In [1]:
import polars as pl
import altair as alt



In [2]:
df = pl.read_csv("../data/memory/memory-sizes.csv")

# Replace 'app' and 'unspecified' (case-insensitive) with 'Runtime'

# first group accross categories 
# then group accross types (.text, .data, .bss)
df.filter(pl.col("board") == "adafruit-feather-nrf52840-sense") \
  .group_by(["environment", "benchmark", "type"]) \
  .agg(pl.sum("size"), pl.first("section_total_size")) \
  .group_by(["environment", "benchmark"]) \
  .agg(pl.sum("size").alias("cosy size"), pl.sum("section_total_size").alias("raw size")) \
  .with_columns((pl.col("cosy size") / pl.col("raw size")).alias("cosy to raw ratio")) \
  .sort("cosy to raw ratio", descending=True)


environment,benchmark,cosy size,raw size,cosy to raw ratio
str,str,i64,i64,f64
"""femto-container""","""xgboost""",42290,32080,1.318267
"""femto-container""","""libud""",37394,33232,1.125241
"""wamr-fast""","""tarfind""",122991,110008,1.118019
"""wamr-fast""","""libud""",123527,110544,1.117446
"""wamr-fast""","""crc_32""",123719,110736,1.117243
…,…,…,…,…
"""micro-bpf""","""libud""",114870,110040,1.043893
"""micropython""","""xgboost""",280710,269228,1.042648
"""micro-bpf""","""xgboost""",121758,116928,1.041307


In [3]:

df.filter(pl.col("board") == "adafruit-feather-nrf52840-sense") \
  .filter(pl.col("environment") == "lua", pl.col("benchmark") == "crc_32") \
  .group_by(["environment", "benchmark", "type"]) \
  .agg(pl.sum("size"), pl.first("section_total_size")) \
  .with_columns((pl.col("size") / pl.col("section_total_size")).alias("ratio"))

environment,benchmark,type,size,section_total_size,ratio
str,str,str,i64,i64,f64
"""lua""","""crc_32""",""".text""",178188,164724,1.081737
"""lua""","""crc_32""",""".bss""",66508,65996,1.007758
"""lua""","""crc_32""",""".data""",624,null,null


In [ ]:
df = df.with_columns(
    pl.when(pl.col("category").str.to_lowercase().is_in(["unspecified", "fill", "app"]))
      .then(pl.lit("Runtime"))
      .otherwise(pl.col("category"))
      .alias("category")
).group_by(["board", "environment", "benchmark", "category", "type"]) \
  .agg(pl.sum("size"), pl.first("total_size"), pl.first("section_total_size")) \
.filter(pl.col("board") != "native64")

df

board,environment,benchmark,category,type,size,total_size,section_total_size
str,str,str,str,str,i64,i64,i64
"""adafruit-feather-nrf52840-sens…","""wamr""","""crc_32""","""Runtime""",""".data""",938,3011079,null
"""adafruit-feather-nrf52840-sens…","""lua""","""tarfind""","""Kernel""",""".data""",622,3503515,null
"""adafruit-feather-nrf52840-sens…","""lua""","""tarfind""","""Kernel""",""".bss""",6499,3503515,65996
"""adafruit-feather-nrf52840-sens…","""femto-container""","""xgboost""","""Application""",""".text""",1186,1916564,27188
"""adafruit-feather-nrf52840-sens…","""femto-container""","""xgboost""","""Kernel""",""".bss""",4879,1916564,4892
…,…,…,…,…,…,…,…
"""adafruit-feather-nrf52840-sens…","""jerryscript""","""xgboost""","""Kernel""",""".data""",158,3474268,null
"""adafruit-feather-nrf52840-sens…","""micropython""","""md5""","""Heap""",""".bss""",15000,3596722,22380
"""adafruit-feather-nrf52840-sens…","""jerryscript""","""tarfind""","""Kernel""",""".bss""",12479,4162750,22332


In [5]:
HEAP_BENCHMARKS = ["md5", "tarfind"]
HEAP_LESS_BENCHMARKS = ["crc_32", "libud", "xgboost"]

DYNAMIC_VMs = ["jerryscript", "micropython", "lua"]
DYNAMIC_VM_CATEGORY_NAME = "Dynamic Managed VMs" 
STATIC_VMs = ["micro-bpf", "femto-container", "wamr", "wamr-fast"]
STATIC_VM_CATEGORY_NAME = "Static Low-level VMs"

RFC_7228_RAM = {
    "Class 1": 10 * 1024,
    "Class 2": 50 * 1024,
}

RFC_7228_ROM = {
    "Class 1": 100 * 1024,
    "Class 2": 250 * 1024,
}

BYTE_LABEL_EXPR = """
(datum.value >= 1024) ? format(datum.value / 1024, '.0f') + ' KiB' : 
format(datum.value, '.0f') + ' B'
"""

sort = ["jerryscript", "micropython", "lua", "micro-bpf", "femto-container", "wamr", "wamr-fast"]
category_order = ["Kernel", "Runtime", "app", "Application",]


colors_category = {
    "Kernel": "#1f77b4",
    "Runtime": "#ff7f0e",
    "Application": "#2ca02c",
    "Heap": "#d62728",
}

df = df.with_columns(
    pl.when(pl.col("environment").is_in(DYNAMIC_VMs)).then(pl.lit(DYNAMIC_VM_CATEGORY_NAME))
    .when(pl.col("environment").is_in(STATIC_VMs)).then(pl.lit(STATIC_VM_CATEGORY_NAME))
    .otherwise(pl.lit(""))
    .alias("vm_category")
)

colors_category_scale = alt.Scale(domain=list(colors_category.keys()), range=list(colors_category.values()))

Y_MAX = 350_000

def rom_chart(data, include_y_axis=True, title="", legend=True):
    y_axis_values = [50 * 1024 * i for i in range(7)]

    y_axis = alt.Axis(labelExpr=BYTE_LABEL_EXPR, title='Size (bytes)', values=y_axis_values) if include_y_axis else alt.Axis(labels=False, title="", values=y_axis_values)
    color_legend = alt.Legend(values=["Kernel", "Runtime", "Application"]) if legend else None

    base_chart = alt.Chart(data)

    bar_chart = base_chart \
        .mark_bar(dx=100) \
        .encode(
            x=alt.X('environment:N', title='', sort=sort),
            y=alt.Y('size:Q', stack='zero', axis=y_axis, scale=alt.Scale(domain=[0, Y_MAX])),
            color=alt.Color('category:N', title='', sort=category_order, scale=colors_category_scale, legend=color_legend),
            order=alt.Order("color_category_sort_index:Q"),
            tooltip=[
                alt.Tooltip('type:N', title='Section'),
                alt.Tooltip('category:N'),
                alt.Tooltip('size:Q', title='Binary size (bytes)')
            ]
        )
    
    return (bar_chart) \
        .transform_calculate(vm_category="split(datum.vm_category, ' ')")  \
        .facet(
            column=alt.Column('vm_category:N', title=title, header=alt.Header(labelBaseline="line-top"), sort=[DYNAMIC_VM_CATEGORY_NAME, STATIC_VM_CATEGORY_NAME]),
            spacing=8
        ).resolve_scale(x='independent')


rom_data = df.filter(pl.col("type").is_in([".text", ".data"]))

crc_32 = rom_data.filter(pl.col("benchmark").eq("crc_32"))
libud = rom_data.filter(pl.col("benchmark").eq("libud"))
xgboost = rom_data.filter(pl.col("benchmark").eq("xgboost"))

print("Static memory usage (ROM) includes .text and .data sections")
alt.hconcat(
    rom_chart(crc_32, title="crc_32"),
    rom_chart(libud, title="libud", include_y_axis=False),
    rom_chart(xgboost, title="xgboost", include_y_axis=False),
    spacing=24
).display(scaleFactor=2)

heap_data = rom_data.filter(pl.col("benchmark").is_in(HEAP_BENCHMARKS))
heap_less_data = rom_data.filter(pl.col("benchmark").is_in(HEAP_LESS_BENCHMARKS))

md5 = rom_data.filter(pl.col("benchmark").eq("md5"))
tarfind = rom_data.filter(pl.col("benchmark").eq("tarfind"))

alt.hconcat(
    rom_chart(md5, title="md5", legend=False),
    rom_chart(tarfind, title="tarfind", include_y_axis=False, legend=False),
    spacing=24
).display(scaleFactor=2)

# rom_chart(heap_less_data).display(scaleFactor=2)
# # .display(scaleFactor=2)
# rom_chart(heap_data).display(scaleFactor=2)


Static memory usage (ROM) includes .text and .data sections


alt.HConcatChart(...)

alt.HConcatChart(...)

In [6]:
chart = (alt.Chart(
        df.filter(pl.col("category") == "Application"))
        .mark_bar()
        .encode(
            x=alt.X('environment:N', title='Environment', sort=sort),
            y=alt.Y('size:Q', stack='zero', title='Size (bytes)', axis=alt.Axis(labelExpr=BYTE_LABEL_EXPR)),
            color=alt.Color('environment:N', title='Environment'),
            tooltip=[
                alt.Tooltip('type:N', title='Section'),
                alt.Tooltip('size:Q', title='Binary size (bytes)')
            ]
        )
        .facet(
            column=alt.Column('benchmark:N', title="Code Size"),
            row = alt.Row('board:N', title="Board")
        )
)

chart.display(scaleFactor=2)

alt.FacetChart(...)

In [7]:
app_code_size = pl.read_csv("../data/memory/application_code_size.csv")
Y_MAX = 28 * 1024

colors = {
    "dynamic":"#1b9e77" ,
    "static": "#d95f02",
    "native": "#7570b3",
}
colors_scale = alt.Scale(domain=list(colors.keys()), range=list(colors.values()))

DYNAMIC_VMs = ["jerryscript", "micropython", "lua"]
STATIC_VMs = ["micro-bpf", "femto-container", "wamr"]


app_code_size = app_code_size.with_columns(
    pl.when(pl.col("environment").is_in(DYNAMIC_VMs)).then(pl.lit(DYNAMIC_VM_CATEGORY_NAME))
    .when(pl.col("environment").is_in(STATIC_VMs)).then(pl.lit(STATIC_VM_CATEGORY_NAME))
    .otherwise(pl.lit("native"))
    .alias("vm_category")
)

app_code_size = app_code_size.with_columns(pl.lit("Executable").alias("Code location"))
femto = app_code_size.filter(pl.col("environment") == "micro-bpf").with_columns(
    (pl.col("size_bytes") - app_code_size.filter(pl.col("environment") == "femto-container").select("size_bytes").to_series()),
    pl.lit("femto-container").alias("environment"),
    pl.lit("Firmware").alias("Code location")
)

app_code_size = pl.concat( [app_code_size, femto])

def plot_code_size(data, title="", include_y_axis=True, legend=True):
    y_axis = alt.Axis(labelExpr=BYTE_LABEL_EXPR, title='Size (bytes)') if include_y_axis else alt.Axis(labels=False,title="")
    color_legend = alt.Legend() if legend else None
     
    return (alt.Chart(data, height=200)
        .mark_bar()
        .transform_calculate(vm_category="split(datum.vm_category, ' ')") 
        .encode(
            x=alt.X('environment:N', title='', sort=sort),
            y=alt.Y('size_bytes:Q', title='Size (bytes)', axis=y_axis, scale=alt.Scale(domain=[0, Y_MAX])),
            color=alt.Color('Code location:N', legend=color_legend, scale=alt.Scale(domain=["Executable", "Firmware"], range=["#1b9e77", "#d95f02"])),
            order=alt.Order("color_category_sort_index:Q"),
            tooltip=[
                alt.Tooltip('size_bytes:Q', title='Binary size (bytes)')
            ]
        )
        .facet(column=alt.Column('vm_category:N', title=title), spacing=8)
        .resolve_scale(x='independent')
)

crc_32 = app_code_size.filter(pl.col("benchmark") == "crc_32")
libud = app_code_size.filter(pl.col("benchmark") == "libud")
xgboost = app_code_size.filter(pl.col("benchmark") == "xgboost")

alt.hconcat(
    plot_code_size(crc_32, title="crc_32"),
    plot_code_size(libud, title="libud", include_y_axis=False),
    plot_code_size(xgboost, title="xgboost", include_y_axis=False),
).display(scaleFactor=2)

md5 = app_code_size.filter(pl.col("benchmark") == "md5")
tarfind = app_code_size.filter(pl.col("benchmark") == "tarfind")

alt.hconcat(
    plot_code_size(md5, title="md5", legend=False),
    plot_code_size(tarfind, title="tarfind", include_y_axis=False, legend=False),
).display(scaleFactor=2)


alt.HConcatChart(...)

alt.HConcatChart(...)

In [8]:
Y_MAX = 240_000

def ram_chart(data, title="", legend=True, include_y_axis=True):
    y_axis_values = [25 * 1024 * i for i in range(10)]
    y_axis = alt.Axis(labelExpr=BYTE_LABEL_EXPR, title='Size (bytes)', values=y_axis_values) if include_y_axis else alt.Axis(labels=False, title="", values=y_axis_values)
    legend = alt.Legend(values=["Kernel", "Runtime", "Heap"]) if legend else None

    return (alt.Chart(data)
        .mark_bar()
        .transform_calculate(vm_category="split(datum.vm_category, ' ')") 
        .encode(
            x=alt.X('environment:N', title='', sort=sort),
            y=alt.Y('size:Q', stack='zero', title='Size (bytes)', axis=y_axis, scale=alt.Scale(domain=[0, Y_MAX])),
            color=alt.Color('category:N', title='', sort=category_order, scale=colors_category_scale, legend=legend),
            order=alt.Order("color_category_sort_index:Q"),
            tooltip=[
                alt.Tooltip('type:N', title='Section'),
                alt.Tooltip('category:N'),
                alt.Tooltip('size:Q', title='Binary size (bytes)')
            ]
        ).facet(
            column=alt.Column('vm_category:N', title=title, sort=[DYNAMIC_VM_CATEGORY_NAME, STATIC_VM_CATEGORY_NAME]), spacing=8
        ).resolve_scale(x='independent')
)

print("Static memory usage (RAM) includes .bss and .data sections.")
ram_data = df.filter(pl.col("type").is_in([".bss", ".data"]))

crc_32 = ram_data.filter(pl.col("benchmark").eq("crc_32"))
libud = ram_data.filter(pl.col("benchmark").eq("libud"))
xgboost = ram_data.filter(pl.col("benchmark").eq("xgboost"))

alt.hconcat(
    ram_chart(crc_32, title="crc_32"),
    ram_chart(libud, title="libud", include_y_axis=False),
    ram_chart(xgboost, title="xgboost", include_y_axis=False),
    spacing=24
).display(scaleFactor=2)

md5 = ram_data.filter(pl.col("benchmark").eq("md5"))
tarfind = ram_data.filter(pl.col("benchmark").eq("tarfind"))

alt.hconcat(
    ram_chart(md5, title="md5", legend=False),
    ram_chart(tarfind, title="tarfind", include_y_axis=False, legend=False),
    spacing=24
).display(scaleFactor=2)

Static memory usage (RAM) includes .bss and .data sections.


alt.HConcatChart(...)

alt.HConcatChart(...)

In [9]:
# df.filter(pl.col("category") == "Heap", pl.col("environment"))